## Stage 1 Trial.

In [9]:
from collections import namedtuple
import os
os.chdir(r"C:\Users\sayan\CNN_Classifier\src")

In [10]:
DataIngestionConfig = namedtuple(typename="DataIngestionConfig",field_names = ["root_dir","source_URL","local_data_file","unzip_dir"])

In [ ]:
## The Entity portion which must be defined in the entity folder
##The dataclass works similarly as nametuple

from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [11]:
## The Config Ingestion portion which must be defined in the config folder of DeepClassifier

from DeepClassifier.constants import *
from DeepClassifier.utils import  read_yaml,create_directories

## All the Configuration Information
class ConfigurationManager:

    
    def __init__(self,
                config_filepath = CONFIG_FILE_PATH,
                params_filepath = PARAMS_FILE_PATH,
                ):

                self.config = read_yaml(config_filepath)
                self.params = read_yaml(params_filepath)
                create_directories([self.config.artifacts_root])

    
    
    def get_data_ingestion_config(self) -> DataIngestionConfig :

        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
                                                    
                                                    root_dir = config.root_dir,
                                                    source_URL= config.source_URL,
                                                    local_data_file = config.local_data_file,
                                                    unzip_dir = config.unzip_dir ,
                                                     
                                                     )

        return data_ingestion_config




In [12]:
## The Data Ingestion Component
import os
import urllib.request as request
from zipfile import ZipFile

class DataIngestion:
    
    def __init__(self, config: DataIngestionConfig):
        
        self.config = config

    ## To download the zip file
    def download_file(self):
        
        if not os.path.exists(self.config.local_data_file):
            
            filename, headers = request.urlretrieve(
                                                   
                                                   url = self.config.source_URL,
                                                   filename = self.config.local_data_file
                                                    
                                                    ) 

    ## This method performs a cleansing action removes unwanted files like files with other extensions
    ## And selects the required files . 
    def _get_updated_list_of_files(self, list_of_files):
        return [file for file in list_of_files if file.endswith(".jpg") and ("Cat" in file or "Dog" in file)]

    
    
    
    ## This extracts the zipfile  and removes files that have file size 0 kb.
    def _preprocess(self, zf: ZipFile, file: str, working_dir: str):
        
        target_filepath = os.path.join(working_dir, file)
        
        if not os.path.exists(target_filepath):
            zf.extract(file, working_dir)

        if os.path.getsize(target_filepath) == 0:
            os.remove(target_filepath)


    
    ## To unzip and extract the downloaded zip file and perform cleansing action to remove unwanted files.
    def unzip_and_clean(self):
        
        with ZipFile(file = self.config.local_data_file, mode="r") as zf: ## zf means zipfile
            
            list_of_files = zf.namelist()
            
            updated_list_of_files = self._get_updated_list_of_files(list_of_files)
            
            for file in updated_list_of_files:
                self._preprocess(zf, file, self.config.unzip_dir)

       

            

In [15]:
os.chdir(r"C:\Users\sayan\CNN_Classifier")

try:
    
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config = data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.unzip_and_clean()

except Exception as e:
    raise e